## Importações e inicialização do Spark

In [ ]:
import os

# coloca as variáveis de ambiente necessárias para o PySpark no windows
os.environ['JAVA_HOME'] = r"C:\Program Files\Eclipse Adoptium\jdk-8.0.452.9-hotspot"
os.environ['SPARK_HOME'] = r"C:\spark\spark-3.5.6-bin-hadoop3"

os.environ['PYSPARK_PYTHON'] = r"C:\Users\diego\Desktop\spark_alura\.venv\Scripts\python.exe"
os.environ['PYSPARK_DRIVER_PYTHON'] = r"C:\Users\diego\Desktop\spark_alura\.venv\Scripts\python.exe"


In [ ]:
import findspark # busca o PySpark no sistema
findspark.init()

In [ ]:
# importa a classe que inicia uma sessão do Spark para usar o Spark SQL
from pyspark.sql import SparkSession

In [ ]:
# inicia uma sessão do Spark, que é o ponto de entrada para usar o Spark SQL
spark = SparkSession.builder.master('local[*]').appName("Iniciando").config("spark.hadoop.io.native.lib", "false").getOrCreate()
spark

## Exemplos de DataFrames

In [ ]:
# exemplo de criação de um DataFrame usando tuplas
data = [('zeca', '35'), ('eva', '29')]
col_names = ['nome', 'idade']

df = spark.createDataFrame(data, col_names)

df.show()

In [ ]:
# exemplo de criação de um DataFrame usando dicionários
data_dict = [{'nome': 'zeca', 'idade': '35'}, {'nome': 'eva', 'idade': '29'}]

df = spark.createDataFrame(data_dict)

df.show()

In [ ]:
# converte o DataFrame do Spark para um DataFrame do Pandas
df.toPandas()

## Extrair conjunto de dados

In [ ]:
zip_path = os.path.abspath(r'.\zip_files')
data_path = os.path.abspath(r'.\data')

In [ ]:
import zipfile # biblioteca nativa do Python para manipular arquivos zip

zipfile.ZipFile(os.path.join(zip_path, 'empresas.zip'), 'r').extractall(data_path)
zipfile.ZipFile(os.path.join(zip_path, 'estabelecimentos.zip'), 'r').extractall(data_path)
zipfile.ZipFile(os.path.join(zip_path, 'socios.zip'), 'r').extractall(data_path)

In [ ]:
# retorna uma lista com os caminhos dos arquivos de um diretório
def get_files_from_directory(directory):
    files = [
        os.path.join(directory, f)
        for f in os.listdir(directory)
    ]
    return files

In [ ]:
# lê todos os arquivos CSV do diretório especificado e cria um DataFrame
df_emps = spark.read.csv(
    get_files_from_directory(os.path.join(data_path, 'empresas')), 
    sep=";", header=False, inferSchema=True)

df_emps.count()

In [ ]:
df_estabs = spark.read.csv(
    get_files_from_directory(os.path.join(data_path, 'estabelecimentos')), 
    sep=";", header=False, inferSchema=True)

df_estabs.count()

In [ ]:
df_socios = spark.read.csv(
    get_files_from_directory(os.path.join(data_path, 'socios')), 
    sep=";", header=False, inferSchema=True)

df_socios.count()

In [ ]:
# exibe as primeiras 3 linhas dos DataFrames
df_emps.show(3)
df_estabs.show(3)
df_socios.show(3)

In [ ]:
# converte as primeiras 3 linhas do DataFrame do Spark para um DataFrame do Pandas
df_socios.limit(3).toPandas()

## Renomear colunas

In [ ]:
# define os nomes das colunas para os DataFrames

emps_col_names = ['cnpj_basico', 'razao_social_nome_empresarial', 'natureza_juridica', 'qualificacao_do_responsavel', 'capital_social_da_empresa', 'porte_da_empresa', 'ente_federativo_responsavel']

estabs_col_names = ['cnpj_basico', 'cnpj_ordem', 'cnpj_dv', 'identificador_matriz_filial', 'nome_fantasia', 'situacao_cadastral', 'data_situacao_cadastral', 'motivo_situacao_cadastral', 'nome_da_cidade_no_exterior', 'pais', 'data_de_inicio_atividade', 'cnae_fiscal_principal', 'cnae_fiscal_secundaria', 'tipo_de_logradouro', 'logradouro', 'numero', 'complemento', 'bairro', 'cep', 'uf', 'municipio', 'ddd_1', 'telefone_1', 'ddd_2', 'telefone_2', 'ddd_do_fax', 'fax', 'correio_eletronico', 'situacao_especial', 'data_da_situacao_especial']

socios_col_names = ['cnpj_basico', 'identificador_de_socio', 'nome_do_socio_ou_razao_social', 'cnpj_ou_cpf_do_socio', 'qualificacao_do_socio', 'data_de_entrada_sociedade', 'pais', 'representante_legal', 'nome_do_representante', 'qualificacao_do_representante_legal', 'faixa_etaria']

In [ ]:
# exibe os nomes das colunas dos DataFrames em formato enumerado para mapeamento das colunas quando renomear
for i, col_name in enumerate(emps_col_names):
    print([i, col_name])

In [ ]:
# função para renomear as colunas de um DataFrame utilizando o mapeamento de nomes de colunas
def rename_columns(df, col_names):
    for i, col_name in enumerate(col_names):
        df = df.withColumnRenamed(f'_c{i}', col_name)
    return df

In [ ]:
# renomeia as colunas dos DataFrames
df_emps = rename_columns(df_emps, emps_col_names)
df_estabs = rename_columns(df_estabs, estabs_col_names)
df_socios = rename_columns(df_socios, socios_col_names)

In [ ]:
# exibe as primeiras 3 linhas dos DataFrames com os nomes das colunas renomeados
print(df_emps.columns)
print(df_estabs.columns)
print(df_socios.columns)

In [ ]:
# converte as primeiras 3 linhas do DataFrame do Spark para DataFrame do Pandas com o nomes das colunas renomeados
df_socios.limit(3).toPandas()

## Converter para coluna de dinheiro para tipo Double

In [ ]:
# mostra os nomes das colunas e seus tipos de dados
df_emps.printSchema()
df_socios.printSchema()
df_estabs.printSchema()

In [ ]:
# importa a classe DoubleType para converter o tipo de dado para Double
# importa a classe functions do para manipulação de colunas
from pyspark.sql.types import DoubleType
from pyspark.sql import functions as fx


In [ ]:
df_emps.show(3)

In [ ]:
# substitui a vírgula por ponto na coluna 'capital_social_da_empresa' para que o PySpark reconheça como número decimal
df_emps = df_emps.withColumn('capital_social_da_empresa', fx.regexp_replace('capital_social_da_empresa', ',', '.'))
df_emps.select('capital_social_da_empresa').show(3)

In [ ]:
df_emps.printSchema()

In [ ]:
# converte a coluna 'capital_social_da_empresa' para o tipo Double
df_emps = df_emps.withColumn('capital_social_da_empresa', df_emps['capital_social_da_empresa'].cast(DoubleType()))
df_emps.printSchema()

## Converter coluna para o tipo Date

In [ ]:
# importa a classe StringType para converter o tipo de dado para String
from pyspark.sql.types import StringType

In [ ]:
# a coluna estava com tipo de dado Integer, mas foi convertida para String para que seja possivel passar to_date
df_socios = df_socios.withColumn('data_de_entrada_sociedade', df_socios['data_de_entrada_sociedade'].cast(StringType()))
df_socios.printSchema()

In [ ]:
df_socios.select('data_de_entrada_sociedade').show(3)

In [ ]:
# para converter para o tipo Date é necessario converter para String primeiro, depois converte para Date usando o mesmo formato de data na coluna original
df_socios = df_socios.withColumn('data_de_entrada_sociedade', fx.to_date(df_socios['data_de_entrada_sociedade'], 'yyyyMMdd'))

df_socios.printSchema()

In [ ]:
df_socios.show(3)

## Converter multiplas colunas em tipo Date

In [ ]:
df_estabs.select('data_situacao_cadastral', 'data_de_inicio_atividade', 'data_da_situacao_especial').show(3)

In [ ]:
# aplica a conversão de data para várias colunas do DataFrame de estabelecimentos
df_estabs = df_estabs\
    .withColumn('data_situacao_cadastral', fx.to_date(df_estabs.data_situacao_cadastral\
        .cast(StringType()), 'yyyyMMdd'))\
    .withColumn('data_de_inicio_atividade', fx.to_date(df_estabs.data_de_inicio_atividade\
        .cast(StringType()), 'yyyyMMdd'))\
    .withColumn('data_da_situacao_especial', fx.to_date(df_estabs.data_da_situacao_especial\
        .cast(StringType()), 'yyyyMMdd'))

df_estabs.select('data_situacao_cadastral', 'data_de_inicio_atividade', 'data_da_situacao_especial').show(3)

In [ ]:
df_estabs.select('data_situacao_cadastral', 'data_de_inicio_atividade', 'data_da_situacao_especial').printSchema()

## Extrair ano de uma coluna de data

In [ ]:
# assim como no SQL, podemos aplicar funções de transformação nas colunas
df_socios\
    .select(
        'nome_do_socio_ou_razao_social',
        fx.year('data_de_entrada_sociedade').alias('ano_entrada_sociedade'), # extrai o ano da data e renomeia a coluna
    )\
    .show(5, truncate=False) # 'truncate=False' para mostrar o texto completo das colunas

### Exercício - Extrair parte de string de uma coluna:
- Criar coluna com ultimo sobrenome e primeiro nome, separados por virgula

In [ ]:
data = [
    ('GISELLE PAULA GUIMARAES CASTRO', 15),
    ('ELAINE GARCIA DE OLIVEIRA', 22),
    ('JOAO CARLOS ABNER DE LOURDES', 43),
    ('MARTA ZELI FERREIRA', 24),
    ('LAUDENETE WIGGERS ROEDER', 51)
]
col_names = ['nome', 'idade']

df = spark.createDataFrame(data, col_names)
df.show(truncate=False)

In [ ]:
# cria uma coluna com o último sobrenome e o primeiro nome, separados por vírgula
df\
    .select(
        'nome',
        fx.concat_ws( # concatena as colunas com um separador
            ', ', 
            fx.substring_index('nome', ' ', -1), # pega o ultimo sobrenome
            fx.substring_index('nome', ' ', 1) # pega o primeiro nome
        ).alias('ident'), 
        'idade'
    )\
    .show(truncate=False)

## Diferença entre NaN, None, Null para o Pandas e o PySpark
- para o PySpark, NaN aparece apenas para o tipo Float, Null aparece para todos os tipos de dados
- para o Pandas, NaN aparece para todos os tipos de dados exceto para o tipo String, que aparece como None

In [ ]:
from pyspark.sql.types import FloatType, IntegerType, StringType, StructType, StructField

In [ ]:
data = [
    (float('nan'), None, None, None),
    (float('nan'), None, None, None),
    (float('nan'), None, None, None),
]

# determina o nome e o tipo de cada coluna do DataFrame
schema = \
    StructType([
        StructField(name="float_nan", dataType=FloatType(), nullable=True),
        StructField(name="float_none", dataType=FloatType(), nullable=True),
        StructField(name="integer_none", dataType=IntegerType(), nullable=True),
        StructField(name="string_none", dataType=StringType(), nullable=True),
    ])

df = spark.createDataFrame(data=data, schema=schema)

df.show(truncate=False)
df.printSchema()

# para o PySpark, NaN aparece apenas para o tipo Float, Null aparece para todos os tipos de dados

In [ ]:
df.toPandas()

# para o Pandas, NaN aparece para todos os tipos de dados exceto para o tipo String, que aparece como None

## Substituir valores Null de tipos diferentes
- ao preencher os nulls com 0 (int), o PySpark troca apenas os valores da coluna com tipo numérico
- ao preencher os nulls com '0' (str), o PySpark troca apenas os valores da coluna com tipo String

In [ ]:
# as colunas 'pais' e 'nome_do_representante' possui valores Null e são de tipos diferentes

df_socios.select('pais', 'nome_do_representante').show(5, truncate=False)

df_socios.select('pais', 'nome_do_representante').printSchema()

df_socios.select('pais', 'nome_do_representante').limit(5).toPandas()

In [ ]:
# contagem de valores Null para colunas específicas
df_socios\
    .select(
    
    fx.count( # conta o número de linhas quando a condição de ser Null é atendida
        fx.when(
            fx.isnull(col='pais'), 1
        )
    ).alias('pais_null_count'),
    
    fx.count(
        fx.when(
            fx.isnull(col='nome_do_representante'), 1
        )
    ).alias('nome_do_representante_null_count')
    
).show()

In [ ]:
# contagem de valores Null em todas as colunas do DataFrame usando lista
df_socios\
    .select(
    [
        fx.count( # conta o número de linhas quando a condição de ser Null é atendida para cada coluna
            fx.when(
                fx.isnull(col_name), 1
            )
        )\
        .alias(col_name[:10]) 
        
        for col_name in df_socios.columns
    ]
).show()


In [ ]:
# ao preencher os nulls com 0 (int), o PySpark troca apenas os valores da coluna com tipo numérico
df_socios.na.fill(0).select('pais', 'nome_do_representante').show(5)

# ao preencher os nulls com '0' (str), o PySpark troca apenas os valores da coluna com tipo String
df_socios.na.fill('0').select('pais', 'nome_do_representante').show(5) 

## Ordenar colunas

In [ ]:
df_socios\
    .select(
        'nome_do_socio_ou_razao_social', 'data_de_entrada_sociedade'
    )\
    .orderBy(
        'data_de_entrada_sociedade', ascending=True
    )\
    .show(5, truncate=False)

In [ ]:
# atribui uma lista de colunas para o método orderBy, permitindo ordenar por múltiplas colunas
df_socios\
    .select(
        'nome_do_socio_ou_razao_social', 'data_de_entrada_sociedade'
    )\
    .orderBy(
        ['data_de_entrada_sociedade', 'nome_do_socio_ou_razao_social'], 
        ascending=[True, True]
    )\
    .show(5, truncate=False)

### Exercício - Ordenar colunas:
- Ordenar o ano e mês dos alunos mais novos para os mais velhos

In [ ]:
data = [
    ('CARMINA RABELO', 4, 2010), 
    ('HERONDINA PEREIRA', 6, 2009), 
    ('IRANI DOS SANTOS', 12, 2010), 
    ('JOAO BOSCO DA FONSECA', 3, 2009), 
    ('CARLITO SOUZA', 1, 2010), 
    ('WALTER DIAS', 9, 2009), 
    ('BRENO VENTUROSO', 1, 2009), 
    ('ADELINA TEIXEIRA', 5, 2009), 
    ('ELIO SILVA', 7, 2010), 
    ('DENIS FONSECA', 6, 2010)
]
col_names = ['nome', 'mes', 'ano']

df = spark.createDataFrame(data, col_names)
df.show(truncate=False)

In [ ]:
# ordena por ano e mês, ambos em ordem decrescente, para que os meses mais recentes apareçam primeiro
df\
    .orderBy(['ano', 'mes'], ascending=[False, False])\
    .show(truncate=False)

## Filtrar dados
- Os métodos `.where` e `.filter` são equivalentes, e filtram da mesma forma os DataFrames
- O método `.like` permite filtrar por parte da string utilizando `%`

In [ ]:
# filtra os dados utilizando o método where
df_emps\
    .select(df_emps.razao_social_nome_empresarial, df_emps.capital_social_da_empresa)\
    .where(df_emps.capital_social_da_empresa == 50)\
    .show(5, truncate=False)

In [ ]:
# filtra os dados utilizando o método filter
df_socios\
    .select(df_socios.nome_do_socio_ou_razao_social)\
    .filter(df_socios.nome_do_socio_ou_razao_social.startswith('DIEGO'))\
    .filter(df_socios.nome_do_socio_ou_razao_social.endswith('CARVALHO'))\
    .show(5, truncate=False)

In [ ]:
# filtra os dados utilizando a função upper para tornar a busca case-insensitive
df_emps\
    .select(df_emps.razao_social_nome_empresarial)\
    .where(fx.upper(df_emps.razao_social_nome_empresarial).like('%RESTAURANTE%'))\
    .show(5, truncate=False)

### Exercício 1 - Filtrar dados
- Realizar a seleção de apenas alunos nascidos no primeiro semestre de 2009 usado os métodos where e filter

In [ ]:
data = [
    ('CARMINA RABELO', 4, 2010), 
    ('HERONDINA PEREIRA', 6, 2009), 
    ('IRANI DOS SANTOS', 12, 2010), 
    ('JOAO BOSCO DA FONSECA', 3, 2009), 
    ('CARLITO SOUZA', 1, 2010), 
    ('WALTER DIAS', 9, 2009), 
    ('BRENO VENTUROSO', 1, 2009), 
    ('ADELINA TEIXEIRA', 5, 2009), 
    ('ELIO SILVA', 7, 2010), 
    ('DENIS FONSECA', 6, 2010)
]
col_names = ['nome', 'mes', 'ano']

df = spark.createDataFrame(data=data, schema=col_names)
df.show(truncate=False)

In [ ]:
# todas as formas abaixo são equivalentes e retornam o mesmo resultado

df\
    .where("ano = 2009")\
    .filter(df.mes < 7)\
    .show(truncate=False)

df\
    .where(df.ano == 2009)\
    .filter("mes < 7")\
    .show(truncate=False)

df\
    .filter("ano == 2009")\
    .where("mes < 7")\
    .show(truncate=False)

In [ ]:
# a estrutura abaixo é considerada como boa prática, pois é mais facil e ser validada pela IDE
df\
    .filter((df.ano == 2009) & (df.mes < 7))\
    .show(truncate=False)

### Exercício 2 - Filtrar dados
- Selecionar apenas os alunos que têm seus nomes iniciados com a letra “C” usando o método like

In [ ]:
data = [
    ('CARMINA RABELO', 4, 2010), 
    ('HERONDINA PEREIRA', 6, 2009), 
    ('IRANI DOS SANTOS', 12, 2010), 
    ('JOAO BOSCO DA FONSECA', 3, 2009), 
    ('CARLITO SOUZA', 1, 2010), 
    ('WALTER DIAS', 9, 2009), 
    ('BRENO VENTUROSO', 1, 2009), 
    ('ADELINA TEIXEIRA', 5, 2009), 
    ('ELIO SILVA', 7, 2010), 
    ('DENIS FONSECA', 6, 2010)
]
col_names = ['nome', 'mes', 'ano']

df = spark.createDataFrame(data, col_names)
df.show(truncate=False)

In [ ]:
df\
    .filter(df.nome.like('C%'))\
    .show(truncate=False)

### Exercício 3 - Filtrar dados
- Aplicar filtro com case-insensitive

In [ ]:
data = [
    ('DIEGO PARAGUAI', 0), 
    ('diego paraguai', 1), 
    ('Diego Paraguai', 2), 
    ('DiegO ParaguaI', 3), 
    ('DieGo PaRaGuai', 4)
]
col_names = ['meu_nome', 'index']

df = spark.createDataFrame(data, col_names)
df.show(truncate=False)

In [ ]:
df\
    .where(fx.upper(df.meu_nome) == 'DIEGO PARAGUAI')\
    .show(truncate=False)

## Sumarizar / Agrupar / Agregar dados
- Sumarizar = resumir dados (estatística)
- Agrupar = juntar linhas em comum de uma coluna
- Agregar = aplicar função para cada linha agrupada (soma, média etc.)

In [ ]:
# agrupa pela contagem de 'ano_de_entrada_sociedade' segmentado por 'ano_de_entrada_sociedade'
df_socios\
    .select(fx.year(df_socios.data_de_entrada_sociedade).alias('ano_de_entrada_sociedade'))\
    .where('ano_de_entrada_sociedade >= 2010')\
    .groupBy('ano_de_entrada_sociedade')\
    .count()\
    .orderBy('ano_de_entrada_sociedade', ascending=True)\
    .show()

In [ ]:
# agrupa pela media de 'capital_social_da_empresa' e a contagem de 'cnpj_basico' segmentado por 'porte_da_empresa'
# '.agg()' é uma forma de realizar vários agrupamentos de uma única vez
df_emps\
    .select(df_emps.cnpj_basico, df_emps.porte_da_empresa, df_emps.capital_social_da_empresa)\
    .groupBy(df_emps.porte_da_empresa)\
    .agg(
        fx.avg(df_emps.capital_social_da_empresa),
        fx.count(df_emps.cnpj_basico)
    )\
    .orderBy(df_emps.porte_da_empresa, ascending=True)\
    .show()

In [ ]:
# mostra as estatisticas gerais (parecido com o método 'describe()' do Pandas)
df_emps\
    .select(df_emps.capital_social_da_empresa)\
    .summary()\
    .show()

df_socios\
    .select(df_socios.faixa_etaria)\
    .summary('count', 'mean')\
    .show()

# count, mean, stddev, min, 25%, 50%, 75%, max

### Exercício - Sumarizar / Agrupar / Agregar dados

In [ ]:
data = [
    ('CARLOS', 'MATEMÁTICA', 7), 
    ('IVO', 'MATEMÁTICA', 9), 
    ('MÁRCIA', 'MATEMÁTICA', 8), 
    ('LEILA', 'MATEMÁTICA', 9), 
    ('BRENO', 'MATEMÁTICA', 7), 
    ('LETÍCIA', 'MATEMÁTICA', 8), 
    ('CARLOS', 'FÍSICA', 2), 
    ('IVO', 'FÍSICA', 8), 
    ('MÁRCIA', 'FÍSICA', 10), 
    ('LEILA', 'FÍSICA', 9), 
    ('BRENO', 'FÍSICA', 1), 
    ('LETÍCIA', 'FÍSICA', 6), 
    ('CARLOS', 'QUÍMICA', 10), 
    ('IVO', 'QUÍMICA', 8), 
    ('MÁRCIA', 'QUÍMICA', 1), 
    ('LEILA', 'QUÍMICA', 10), 
    ('BRENO', 'QUÍMICA', 7), 
    ('LETÍCIA', 'QUÍMICA', 9)
]
col_names = ['nome', 'materia', 'nota']

df = spark.createDataFrame(data, col_names)
df.show()

In [ ]:
# cria uma coluna com 'withColumns', com valores baseado no teste feito na coluna 'nota' através do método 'when()' e 'otherwise()'
df = df.withColumn('status', fx.when(df.nota >= 7, "APROVADO").otherwise("REPROVADO"))

df.show()

In [ ]:
# descreve a distribuicao dos dados de nota
df\
    .select('nota')\
    .summary('min', '25%', '50%', '75%', 'max')\
    .show()

# agrupa pela contagem de 'status' segmentado por 'status'
df\
    .groupBy('status')\
    .count()\
    .orderBy('status', ascending=True)\
    .show()

## Juntar DataFrames

In [ ]:
print(df_emps.columns)
print(df_estabs.columns)

In [ ]:
# junta internamente os DataFrames através da coluna 'cnpj_basico'
df_emps_join_estabs = df_emps.join(other=df_estabs, on='cnpj_basico', how='inner')

df_emps_join_estabs.show(5)
print(df_emps_join_estabs.columns)

In [ ]:
# conta o 'cnpj_basico' por 'ano_de_entrada_sociedade' a partir de 2010
# obs.: o método 'sort()' ordenada mais rápido que o 'orderBy()', porém ele pode ordernar errado
df_count_por_ano = \
    df_socios\
        .select(
            fx.year(df_socios.data_de_entrada_sociedade).alias('ano_de_entrada_sociedade'), 
            df_socios.cnpj_basico
        )\
        .filter('ano_de_entrada_sociedade >= 2010')\
        .groupBy('ano_de_entrada_sociedade')\
        .agg(fx.count(df_socios.cnpj_basico).alias('contagem'))\
        .sort('ano_de_entrada_sociedade')
        
df_count_por_ano.show()

In [ ]:
# faz a soma da coluna contagem e cria um valor literal para colocar na coluna 'ano_de_entrada_sociedade'
df_count_por_ano\
    .select(
        fx.lit('Total').alias('ano_de_entrada_sociedade'), # cria um valor literal
        fx.sum(df_count_por_ano.contagem).alias('contagem')
    )\
    .show()

In [ ]:
# une o DataFrame com a contagem por ano com a seleção que faz a contagem total + valor literal
df_count_por_ano\
    .union(
        df_count_por_ano\
        .select(
            fx.lit('Total').alias('ano_de_entrada_sociedade'), 
            fx.sum(df_count_por_ano.contagem).alias('contagem')
        )
    ).show()

### Exercício 1 - Juntar DataFrames

In [ ]:
produtos = spark.createDataFrame(
    [
        ('1', 'Bebidas', 'Água mineral'), 
        ('2', 'Limpeza', 'Sabão em pó'), 
        ('3', 'Frios', 'Queijo'), 
        ('4', 'Bebidas', 'Refrigerante'),
        ('5', 'Pet', 'Ração para cães')
    ],
    ['id', 'cat', 'prod']
)

impostos = spark.createDataFrame(
    [
        ('Bebidas', 0.15), 
        ('Limpeza', 0.05),
        ('Frios', 0.065),
        ('Carnes', 0.08)
    ],
    ['cat', 'tax']
)

In [ ]:
produtos.show()
impostos.show()

In [ ]:
# how = a junção usada por padrão é o inner, mas pode ser um desses: 
# inner, cross, outer, full, fullouter, full_outer, left, leftouter, left_outer, right, rightouter, right_outer, semi, leftsemi, left_semi, anti, leftanti and left_anti
produtos\
    .join(impostos, 'cat', how='outer')\
    .sort(produtos.id)\
    .show()

### Exercício 2 - Juntar DataFrames

In [ ]:
idades = spark.createDataFrame(
    [
        ('CARLOS', 15), 
        ('IVO', 14), 
        ('MÁRCIA', 16), 
        ('LEILA', 17), 
        ('LETÍCIA', 14)
    ],
    ['nomes', 'idades']
)

notas = spark.createDataFrame(
    [
        ('CARLOS', 10), 
        ('MÁRCIA', 1), 
        ('LEILA', 10), 
        ('BRENO', 7), 
        ('LETÍCIA', 9)
    ],
    ['nomes', 'notas']
)

In [ ]:
idades.show()
notas.show()

In [ ]:
idades\
    .join(notas, 'nomes', how='left')\
    .sort('nomes')\
    .show()